In [136]:
# Import Splinter and BeautifulSoup
from splinter import Browser
import pandas as pd
from bs4 import BeautifulSoup as soup
import time
from webdriver_manager.chrome import ChromeDriverManager

In [137]:
# executable_path = {'executable_path': ChromeDriverManager().install()}
# browser = Browser('chrome', **executable_path, headless=False)
executable_path = {'executable_path': ChromeDriverManager(version = '98.0.4758.102').install()}

browser = Browser('chrome', **executable_path, headless=False)



====== WebDriver manager ======
Trying to download new driver from https://chromedriver.storage.googleapis.com/98.0.4758.102/chromedriver_win32.zip
Driver has been saved in cache [C:\Users\tlr20\.wdm\drivers\chromedriver\win32\98.0.4758.102]


Visit the NASA Mars News Site

In [138]:
# Visit the mars nasa news site
url = 'https://redplanetscience.com'
browser.visit(url)
# Optional delay for loading the page
#we're searching for elements with a specific combination of tag (div) and attribute (list_text). As an example, ul.item_list would be found in HTML as <ul class="item_list">.
#Secondly, we're also telling our browser to wait one second before searching for components.
browser.is_element_present_by_css('div.list_text', wait_time=1)

True

In [139]:
# set up the HTML parser:
html = browser.html
news_soup = soup(html, 'html.parser')
slide_elem = news_soup.select_one('div.list_text')

In [140]:
# assign the title and summary text to variables we'll reference later. (.find = this variable holds a ton of info, so look inside of that info to find this)
slide_elem.find('div', class_='content_title')

<div class="content_title">NASA Mars Mission Connects With Bosnian and Herzegovinian Town</div>

In [141]:
# Use the parent element to find the first `a` tag and save it as `news_title`
news_title = slide_elem.find('div', class_='content_title').get_text()
news_title

'NASA Mars Mission Connects With Bosnian and Herzegovinian Town'

In [142]:
# Use the parent element to find the paragraph text
news_p = slide_elem.find('div', class_='article_teaser_body').get_text()
news_p

'A letter from NASA was presented to the mayor of Jezero, Bosnia-Herzegovina, honoring the connection between the town and Jezero Crater, the Mars 2020 rover landing site.'

### Featured Images

In [143]:
# Visit URL
url = 'https://spaceimages-mars.com'
browser.visit(url)

In [144]:
# Find and click the full image button; indexing on first line of code stipulates that we want our browser to click the second button
full_image_elem = browser.find_by_tag('button')[1]
full_image_elem.click()

In [145]:
# Parse the resulting html with soup
html = browser.html
img_soup = soup(html, 'html.parser')

In [146]:
# Find the relative image url; An img tag is nested within this HTML, so we've included it.
# .get('src') pulls the link to the image.
img_url_rel = img_soup.find('img', class_='fancybox-image').get('src')
img_url_rel

'image/featured/mars3.jpg'

In [147]:
# Use the base URL to create an absolute URL
img_url = f'https://spaceimages-mars.com/image/featured/mars2.jpg'
img_url

'https://spaceimages-mars.com/image/featured/mars2.jpg'

Mars Facts

In [148]:
# go to a new website to grab a table; creating a new DataFrame from the HTML table. 
# The Pandas function read_html() specifically searches for and returns a list of tables found in the HTML. 
# By specifying an index of 0, we're telling Pandas to pull only the first table it encounters, or the first item in the list. 
# Then, it turns the table into a DataFrame.
df = pd.read_html('https://galaxyfacts-mars.com')[0]
df.columns=['description', 'Mars', 'Earth']
df.set_index('description', inplace=True)
df

,Mars,Earth
description,,
Mars - Earth Comparison,Mars,Earth
Diameter:,"6,779 km","12,742 km"
Mass:,6.39 × 10^23 kg,5.97 × 10^24 kg
Moons:,2,1
Distance from Sun:,"227,943,824 km","149,598,262 km"
Length of Year:,687 Earth days,365.24 days
Temperature:,-87 to -5 °C,-88 to 58°C


In [149]:
df.to_html()

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>Mars</th>\n      <th>Earth</th>\n    </tr>\n    <tr>\n      <th>description</th>\n      <th></th>\n      <th></th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>Mars - Earth Comparison</th>\n      <td>Mars</td>\n      <td>Earth</td>\n    </tr>\n    <tr>\n      <th>Diameter:</th>\n      <td>6,779 km</td>\n      <td>12,742 km</td>\n    </tr>\n    <tr>\n      <th>Mass:</th>\n      <td>6.39 × 10^23 kg</td>\n      <td>5.97 × 10^24 kg</td>\n    </tr>\n    <tr>\n      <th>Moons:</th>\n      <td>2</td>\n      <td>1</td>\n    </tr>\n    <tr>\n      <th>Distance from Sun:</th>\n      <td>227,943,824 km</td>\n      <td>149,598,262 km</td>\n    </tr>\n    <tr>\n      <th>Length of Year:</th>\n      <td>687 Earth days</td>\n      <td>365.24 days</td>\n    </tr>\n    <tr>\n      <th>Temperature:</th>\n      <td>-87 to -5 °C</td>\n      <td>-88 to 58°C</td>\n    </tr>\n  </tbody>

D1: Scrape High-Resolution Mars’ Hemisphere Images and Titles

Hemispheres

In [150]:
# 1. Use browser to visit the URL 
url = 'https://marshemispheres.com/'
browser.visit(url)

html = browser.html

img_soup = soup(html, 'html.parser')

In [151]:
# Retreive all items that contain mars hemispheres information
parent = img_soup.find_all('div', class_='item')

# 2. Create empty list for hemisphere urls 
hemisphere_image_urls = []

# 3. Write code to retrieve the image urls and titles for each hemisphere.
# 3A. create lists for images and titles 
images = []
titles = []
for item in parent:
    images.append(url + item.find('a')['href'])
    titles.append(item.find('h3').text.strip())
print(images)
titles

['https://marshemispheres.com/cerberus.html', 'https://marshemispheres.com/schiaparelli.html', 'https://marshemispheres.com/syrtis.html', 'https://marshemispheres.com/valles.html']


['Cerberus Hemisphere Enhanced',
 'Schiaparelli Hemisphere Enhanced',
 'Syrtis Major Hemisphere Enhanced',
 'Valles Marineris Hemisphere Enhanced']

In [152]:
# 3B. create full length url template
browser.visit(images[0])
html = browser.html
img_soup = soup(html, 'html.parser')

single_url = url + img_soup.find('img',class_='wide-image')['src']
single_url


'https://marshemispheres.com/images/f5e372a36edfa389625da6d0cc25d905_cerberus_enhanced.tif_full.jpg'

In [153]:
# 3C. create full length url  for each image
img_urls = []
for single_url in images:
    browser.visit(single_url)
    html = browser.html
    img_soup = soup(html, 'html.parser')
#  3D. savetofile
    single_url = url+img_soup.find('img',class_='wide-image')['src']
    img_urls.append(single_url)
    
img_urls

['https://marshemispheres.com/images/f5e372a36edfa389625da6d0cc25d905_cerberus_enhanced.tif_full.jpg',
 'https://marshemispheres.com/images/3778f7b43bbbc89d6e3cfabb3613ba93_schiaparelli_enhanced.tif_full.jpg',
 'https://marshemispheres.com/images/555e6403a6ddd7ba16ddb0e471cadcf7_syrtis_major_enhanced.tif_full.jpg',
 'https://marshemispheres.com/images/b3c7c6c9138f57b4756be9b9c43e3a48_valles_marineris_enhanced.tif_full.jpg']

In [154]:
# 4. Print the list that holds the dictionary of each image url and title.
hemisphere_image_urls = []

for i in range(len(titles)):
    hemisphere_image_urls.append({'title':titles[i],'img_url':img_urls[i]})

hemisphere_image_urls

[{'title': 'Cerberus Hemisphere Enhanced',
  'img_url': 'https://marshemispheres.com/images/f5e372a36edfa389625da6d0cc25d905_cerberus_enhanced.tif_full.jpg'},
 {'title': 'Schiaparelli Hemisphere Enhanced',
  'img_url': 'https://marshemispheres.com/images/3778f7b43bbbc89d6e3cfabb3613ba93_schiaparelli_enhanced.tif_full.jpg'},
 {'title': 'Syrtis Major Hemisphere Enhanced',
  'img_url': 'https://marshemispheres.com/images/555e6403a6ddd7ba16ddb0e471cadcf7_syrtis_major_enhanced.tif_full.jpg'},
 {'title': 'Valles Marineris Hemisphere Enhanced',
  'img_url': 'https://marshemispheres.com/images/b3c7c6c9138f57b4756be9b9c43e3a48_valles_marineris_enhanced.tif_full.jpg'}]

In [155]:
# 4A. Print the individual linked image url and title.
for i in range(len(hemisphere_image_urls)):
    print(hemisphere_image_urls[i]['title'])
    print(hemisphere_image_urls[i]['img_url'] + '\n')

Cerberus Hemisphere Enhanced
https://marshemispheres.com/images/f5e372a36edfa389625da6d0cc25d905_cerberus_enhanced.tif_full.jpg

Schiaparelli Hemisphere Enhanced
https://marshemispheres.com/images/3778f7b43bbbc89d6e3cfabb3613ba93_schiaparelli_enhanced.tif_full.jpg

Syrtis Major Hemisphere Enhanced
https://marshemispheres.com/images/555e6403a6ddd7ba16ddb0e471cadcf7_syrtis_major_enhanced.tif_full.jpg

Valles Marineris Hemisphere Enhanced
https://marshemispheres.com/images/b3c7c6c9138f57b4756be9b9c43e3a48_valles_marineris_enhanced.tif_full.jpg



In [156]:
# 5. Quit the browser
browser.quit()

hemisphere_image_urls

[{'title': 'Cerberus Hemisphere Enhanced',
  'img_url': 'https://marshemispheres.com/images/f5e372a36edfa389625da6d0cc25d905_cerberus_enhanced.tif_full.jpg'},
 {'title': 'Schiaparelli Hemisphere Enhanced',
  'img_url': 'https://marshemispheres.com/images/3778f7b43bbbc89d6e3cfabb3613ba93_schiaparelli_enhanced.tif_full.jpg'},
 {'title': 'Syrtis Major Hemisphere Enhanced',
  'img_url': 'https://marshemispheres.com/images/555e6403a6ddd7ba16ddb0e471cadcf7_syrtis_major_enhanced.tif_full.jpg'},
 {'title': 'Valles Marineris Hemisphere Enhanced',
  'img_url': 'https://marshemispheres.com/images/b3c7c6c9138f57b4756be9b9c43e3a48_valles_marineris_enhanced.tif_full.jpg'}]